# SPARQL queries to RaDiPO ontology

In [2]:
%pip install rdflib

In [51]:
import rdflib
onto = rdflib.Graph()
onto.parse("C:/Users/Usuario/Documents/GitHub/RaDiPO/radipo.owl")

<Graph identifier=N8af268ba886d4c678a98a6a731f4c3a3 (<class 'rdflib.graph.Graph'>)>

**1. Number of disorders (clinical entity)**

In [11]:
query1 = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 
        SELECT (COUNT (DISTINCT ?disorder) AS ?num_disorders)
        WHERE
        {?disorder a radipo:Clinical_entity  .
        }""")

for row in query1:
    print("Number of disorders: ", row.num_disorders)

Number of disorders:  4199


**2. Number of proteins**

In [10]:
query = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 
        SELECT (COUNT (DISTINCT ?protein) AS ?num_proteins)
        WHERE
        {?protein a radipo:Protein  .
        }""")

for row in query:
    print("Number of proteins: ", row.num_proteins)

Number of proteins:  4262


**3. Number of disorders with symptom "Intellectual disability"**

In [35]:
query = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT (COUNT (DISTINCT ?disorder) AS ?num_disorders)
        WHERE
        {?disorder a radipo:Clinical_entity  ;
                radipo:has_symptom ?symptom.
         ?symptom rdf:type radipo:Symptom ;
                radipo:has_symptom_name ?symptom_name .
        FILTER (CONTAINS(?symptom_name, "Intellectual disability"))
        
        }""")
        
for row in query:
    print("Number of disorders with symptom Intellectual disability: ", row.num_disorders)

Number of disorders with symptom Intellectual disability:  772


**4. Ids of enzymes and the catalytic activity**

In [48]:
query = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?id ?activity
        WHERE
        {?protein rdf:type radipo:Enzyme ;
                radipo:has_protein_id ?id ;
                radipo:has_catalytic_activity ?activity .
        } LIMIT 10""")

print("Disorder ID\tEnzyme activity")     
for row in query:
    print(row.id,"\t", row.activity)

Disorder ID	Enzyme activity
O00238 	 Reaction=ATP + L-threonyl-[receptor-protein] = ADP + H(+) + O-phospho-L-threonyl-[receptor-protein]
O00429 	 Reaction=GTP + H2O = GDP + H(+) + phosphate
O14672 	 Reaction=Endopeptidase of broad specificity.
O14832 	 Reaction=2-oxoglutarate + O2 + phytanoyl-CoA = 2-hydroxyphytanoyl-CoA + CO2 + succinate
O60674 	 Reaction=ATP + L-tyrosyl-[protein] = ADP + H(+) + O-phospho-L-tyrosyl-[protein]
O60832 	 Reaction=uridine in 5S rRNA = pseudouridine in 5S rRNA
O95263 	 Reaction=3',5'-cyclic AMP + H2O = AMP + H(+)
P00734 	 Reaction=Selective cleavage of Arg-|-Gly bonds in fibrinogen to form fibrin and release fibrinopeptides A and B.
P01111 	 Reaction=GTP + H2O = GDP + H(+) + phosphate
P01112 	 Reaction=GTP + H2O = GDP + H(+) + phosphate


**5. Drug name, id and phase related to a disorder **

In [69]:
query = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?disorder_name ?drug_name ?drug_id 
        WHERE
        {
            ?disorder a radipo:Clinical_entity ;
                    radipo:is_associated_to_gene ?gene ;
                    radipo:has_disorder_name ?disorder_name .
            ?gene rdf:type radipo:Gene_with_protein ;
                radipo:has_protein_product ?protein .
            ?protein radipo:is_target_of ?drug .
            ?drug radipo:has_drug_name ?drug_name ;
                radipo:has_drug_id ?drug_id .
                
        } LIMIT 10""")

print("Disorder name\tDrug name\tDrug ID")     
for row in query:
        print(row.disorder_name, row.drug_name, row.drug_id)


Disorder name	Drug name	Drug ID
Williams syndrome Rofecoxib DB00533
Early-onset autosomal dominant Alzheimer disease Deferoxamine DB00746
Early-onset autosomal dominant Alzheimer disease Aluminium DB01370
Early-onset autosomal dominant Alzheimer disease Zinc DB01593
Early-onset autosomal dominant Alzheimer disease Methionine Sulfoxide DB02235
Early-onset autosomal dominant Alzheimer disease Resveratrol DB02709
Early-onset autosomal dominant Alzheimer disease Tris(Hydroxymethyl)Aminomethane DB03754
Early-onset autosomal dominant Alzheimer disease Phenserine DB04892
Early-onset autosomal dominant Alzheimer disease Tetrathiomolybdate DB05088
Early-onset autosomal dominant Alzheimer disease Dimercaprol DB06782


We can turn this into a table and use it for searching drugs related to a disease. The drugs are ralated to the protein related to the disorder, so their indications could not be the said disorder. However, there are research lines exploring how if these drugs can be applied for drug therapy in rare diseases too. This information could be a starting point of a research project for looking for drugs for a specific disorder.


**6.Number of consequences/disabilities**

In [70]:
query = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 
        SELECT (COUNT (DISTINCT ?disability) AS ?num_dis)
        WHERE
        {?disability a radipo:Disability  .
        }""")

for row in query:
    print("Number of consequences/disabilities: ", row.num_dis)

Number of consequences/disabilities:  843


**7.Disorders whose patients have affected their writing skills**

In [92]:
query = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 

        SELECT ?name
        WHERE
        {
                ?ability_class a radipo:Ability_affected  ;
                        radipo:ability_specification ?ability .
                FILTER (?ability = "Writing"^^xsd:string) .

                ?disability rdf:type radipo:Disability;
                        radipo:affects_ability ?ability_class .

                ?disorder radipo:is_associated_to_disability ?disability;
                        radipo:has_disorder_name ?name .

                 
        }""")

for row in query:
        print(row.name)


Autosomal dominant optic atrophy plus syndrome
X-linked cerebral adrenoleukodystrophy
Trichothiodystrophy
Infantile spasms syndrome
Treacher-Collins syndrome
WAGR syndrome
Waardenburg syndrome type 1
Waardenburg syndrome type 2
Waardenburg-Shah syndrome
Williams syndrome
Autosomal dominant optic atrophy, classic form
X-linked Emery-Dreifuss muscular dystrophy


**8. Number of disorders with patients that have the skill "Transferring oneself" affected.**

In [102]:
query = onto.query("""
        PREFIX radipo:<https://w3id.org/def/radipo#> 

        SELECT (COUNT(DISTINCT ?disorder) AS ?num_dis)
        WHERE
        {
                ?ability_class a radipo:Ability_affected  ;
                        radipo:ability_specification ?ability .
                FILTER (?ability = "Transferring oneself"^^xsd:string) .

                ?disability rdf:type radipo:Disability;
                        radipo:affects_ability ?ability_class .

                ?disorder radipo:is_associated_to_disability ?disability.
        
        }""")

for row in query:
        print(row.num_dis)

7


Patients with these disability may need physical therapy, which is an information needed by health professionals 